# ELEC3305 - Real Time Sonar App


In [10]:
# Import functions and libraries
import numpy as np
from scipy import signal
from numpy import *
import matplotlib.pyplot as plt
from rtsonar2 import rtsonar
import peakdet

### Task III: Play with different parameters with the real-time sonar!

The function `rtsonar()` in `rtsonar.py` provides a wrapper to create a real-time sonar plot using user-defined processing functions. Copy and paste your functions genPulseTrain(), genChirpPulse(), crossCorr(), findDelay(), and dist2time() to the code cell below. Scroll down to the bottom and you should be able to run `rtsonar()`!

The detection resolution and sensitivity will depend on the parameters you will choose for the sonar. The longer the pulse is, the worse time resolution you will get, but much stronger matched filtering amplitude. You can buy the time resolution by using pulse compression with chirp pulses at the expense of increasing the bandwidth. 

* Run the sonar with different parameters. You can get interesting targets by moving your laptop, moving yourself next to the computer, using a book as a reflecting surface. It would be easier for you to distinguish the target if you move it back and forth. Play with the pulse lengths, the frequency sweeps, etc. 
* To get the best real-time performance, you should restart the kernel every time you run this lab. 
* Submit 4 interesting set of parameters. Explain why you chose them. 




In [11]:
# Copy and paste your 5 functions here:
# genChirpPulse()
def generate_chirp_analytic(f0, f1, T, fs):
    # s_chirp_a = exp(1j* phi_of_t )
    tindex = np.linspace(0, T, round(T * fs))
    phi_of_t = 2 * np.pi * (f0 * tindex + ((f1 - f0)/(2 * T)) * tindex * tindex)

    s_chirp_a = np.exp( 1j* phi_of_t )
    return s_chirp_a

def genChirpPulse(Npulse, f0, f1, fs):
    #     Function generates an analytic function of a chirp pulse
    #     Inputs:
    #             Npulse - pulse length in samples
    #             f0     - starting frequency of chirp
    #             f1     - end frequency of chirp
    #             fs     - sampling frequency
    #     Output:
    #              pulse - chirp pulse
    return generate_chirp_analytic(f0, f1, Npulse/fs, fs)
    



    
# genPulseTrain()
def genPulseTrain(pulse, Nrep, Nseg):
    # Funtion generates a pulse train from a pulse. 
    #Inputs:
    #    pulse = the pulse generated by genChirpPulse
    #    Nrep  =  number of pulse repetitions
    #    Nseg  =  Length of pulse segment >= length(pulse)
    if Nseg < len(pulse):
        raise ValueError("Nseg needs to be bigger than length of pulse")

    return np.tile(np.pad(pulse, (0, Nseg - len(pulse)), 'constant'), Nrep)
    
# crossCorr()
def crossCorr( rcv, pulse_a ):
    # Funtion generates cross-correlation between rcv and pulse_a
    # Inputs:
    #    rcv - received signal
    #    pulse_a - analytic pulse
    # Output:
    #    Xrcv - cross-correlation between rcv and pulse_a
    return signal.correlate(rcv, pulse_a, mode='full')


# findDelay()
def findDelay(Xrcv, Nseg):
    # finds the first pulse
    # Inputs:  
    #         Xrcv - the received matched filtered signal
    #         Nseg - length of a segment
    # Output:
    #          idx - index of the beginning of the first pulse
    data = Xrcv #dont do the above! for the real signal this is bad.
    #pks = signal.find_peaks_cwt(data, 50, noise_perc=30)
    pks, pks2 = peakdet.peakdet(data, 0.3)
    #print(pks)
    return int(pks[0, 0]) if len(pks) > 0 else 0

def temp2speed(T): #speed = meters
    return 331.5 * np.sqrt(1 + T/273.15)

# dist2time()
def dist2time( dist, temperature=21):
    # Converts distance in cm to time in secs
    # Inputs:
    # dist        - distance to object in cm
    # temperature - in celcius
    # Output:
    # t           - time in seconds between transmitted pulse and echo
    dist /= 100
    dist *= 2
    speed = temp2speed(temperature)
    return dist / speed


In [14]:
# Run this for Real-time Sonar
# Change the parameters!
# Your functions will need to be efficient for this to work. 
# If you get nothing try thinking about how to speed up your code.
fs = 48000 # Sampling frequency
f0 = 6000 # Chirp initial frequency
f1 = 12000 # Chirp ending frequency

Npulse = 500 # Length of Chirp Pulse
Nrep = 24 # Number of repetition in a pulse train (determines vertical length of plot )
Nseg = 2048*2 # Number of samples between pulses (determines maximum time-of-arrival)
Nplot = 200 # Number of pixels in plot along horizontal direction (higher resolution is slower)
maxdist = 200 # Maximum distance in cm
temperature = 20 # Temperature     


functions = (genChirpPulse, genPulseTrain, crossCorr, findDelay, dist2time) #join the functions together

#this doesn't work in jupyter notebook ("exception opening new comm")
#lab works though with jupyter_bokeh extension (nice)
stop_flag = rtsonar( f0, f1, fs, Npulse, Nseg, Nrep, Nplot, maxdist, temperature, functions )

[GlyphRenderer(id='p5748', ...)] ColumnDataSource(id='p5742', ...)


In [15]:
# Run this to stop the sonar (it will take a few seconds to stop)
stop_flag.set()